# Question 1

# Libraries

Ok, Firstly, thank you for reviewing my work. each section has a heading and below has details of what is done in that section.

below is a list of libraries I used in completing my project.

In [142]:
import pandas as pd
import numpy as np
import requests
import csv
import matplotlib.pyplot as ply
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium


print('Libraries have been installed')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries have been installed


---------

# Importing the dataframe - Nightmare

I used the Pandas read method below to import the table from the Wiki-page

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)

Checked the dataframe type and length

In [4]:
print(type(df))
print(len(df))

<class 'list'>
3


# Wrangling etc

Some wrangling- changed 'Not assigned' neighborhood to the same as the Borough and dropped the Not assigned Boroughs then a slight table re-format.

In [5]:
df1=df[0]
df1['Neighbourhood']=np.where(df1['Neighbourhood']== 'Not assigned' ,df1['Borough'],df1['Neighbourhood'])
df1.replace('Not assigned', np.nan,inplace=True)
df1.dropna(subset=['Borough'],axis=0,inplace=True)
df1.reset_index(drop=True, inplace=True)
df2=df1.groupby(['Postcode','Borough'],sort=False).agg(lambda x:",".join(x))
df2.reset_index(drop=False, inplace=True)
df2.rename(columns={'Postcode':'Postal Code','Neighbourhood':'Neighborhood'},inplace=True)
df2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


## The shape of my data frame.

In [26]:
df2.shape

(103, 3)

## Tried the Geocoder -  - Um no luck

In [7]:
!pip install geocoder

    100% |████████████████████████████████| 102kB 7.5MB/s ta 0:00:01
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda

In [22]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

Neighborhoods=df2['Postal Code']
Neighborhoods
for i in Neighborhoods:
    g = geocoder.google('{}, Toronto'.format('Post Code'))
    lat_lng_coords = g.latlng
    print(lat_lng_coords)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [9]:
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]


TypeError: 'NoneType' object is not subscriptable

## I imported the Geodata

In [11]:
url2='https://cocl.us/Geospatial_data'
df_lat_lng=pd.read_csv(url2)
df_lat_lng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df3=df2.sort_values(['Postal Code'])

## Next I used the sorted dataframe with postal codes and merged it creating a new data frame

In [159]:
df4=pd.merge(df3,df_lat_lng, on='Postal Code') 
df4.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Created a map to check if i am on the right track.

In [51]:
latitude = 43.741667
longitude=-79.373333

map_tor_can=folium.Map(location=[latitude,longitude],zoom_start=11)

for lat,lng,label in zip(df4['Latitude'],df4['Longitude'],df4['Neighborhood']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                       parse_html=False).add_to(map_tor_can)

map_tor_can

## Re-arranged the table so that i could get a Borough count for each Postal Code

In [96]:
areas_grouped=df4.groupby('Borough').count()
areas_grouped

,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


## Created one hots for each neighborhood.

In [103]:
tor_onehot=pd.get_dummies(df4[['Neighborhood']],prefix="",prefix_sep="")

tor_onehot['Borough']=df4['Borough']

fixed_columns=[tor_onehot.columns[-1]]+ list(tor_onehot.columns[:-1])
tor_onehot=tor_onehot[fixed_columns]

tor_onehot

,Borough,"Adelaide,King,Richmond",Agincourt,"Agincourt North,L'Amoreaux East,Milliken,Steeles East","Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown","Alderwood,Long Branch","Bathurst Manor,Downsview North,Wilson Heights",Bayview Village,"Bedford Park,Lawrence Manor East",Berczy Park,...,Upper Rouge,Victoria Village,Westmount,Weston,Willowdale South,Willowdale West,Woburn,"Woodbine Gardens,Parkview Hill",Woodbine Heights,York Mills West
0,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
tor_onehot.shape

(103, 104)

In [105]:
tor_grouped=tor_onehot.groupby('Borough').mean().reset_index()
tor_grouped

,Borough,"Adelaide,King,Richmond",Agincourt,"Agincourt North,L'Amoreaux East,Milliken,Steeles East","Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown","Alderwood,Long Branch","Bathurst Manor,Downsview North,Wilson Heights",Bayview Village,"Bedford Park,Lawrence Manor East",Berczy Park,...,Upper Rouge,Victoria Village,Westmount,Weston,Willowdale South,Willowdale West,Woburn,"Woodbine Gardens,Parkview Hill",Woodbine Heights,York Mills West
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1,Downtown Toronto,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.2,0.2,0.000000
4,Etobicoke,0.000000,0.000000,0.000000,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.083333,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
6,North York,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.041667,0.041667,0.000000,...,0.000000,0.041667,0.000000,0.0,0.041667,0.041667,0.000000,0.0,0.0,0.041667
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
8,Scarborough,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000
9,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000


## Used KMeans to cluster the neighborhoods

In [111]:
Kclusters=11

Tor_group_clus= tor_grouped.drop('Borough',1)

kmeans=KMeans(n_clusters=Kclusters,random_state=0).fit(Tor_group_clus)
kmeans.labels_[0:10]

array([ 8,  9,  7,  6,  2,  0,  4,  3, 10,  1], dtype=int32)

In [126]:
#tor_grouped.insert(0,'Cluster Labels',kmeans.labels_)
tor_merged=df4
tor_merged=tor_merged.join(tor_grouped.set_index('Borough'),on='Borough')

tor_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,"Adelaide,King,Richmond",Agincourt,"Agincourt North,L'Amoreaux East,Milliken,Steeles East","Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",...,Upper Rouge,Victoria Village,Westmount,Weston,Willowdale South,Willowdale West,Woburn,"Woodbine Gardens,Parkview Hill",Woodbine Heights,York Mills West
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,10,0.000000,0.058824,0.058824,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,10,0.000000,0.058824,0.058824,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,10,0.000000,0.058824,0.058824,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000
3,M1G,Scarborough,Woburn,43.770992,-79.216917,10,0.000000,0.058824,0.058824,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,10,0.000000,0.058824,0.058824,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,10,0.000000,0.058824,0.058824,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,10,0.000000,0.058824,0.058824,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,10,0.000000,0.058824,0.058824,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,10,0.000000,0.058824,0.058824,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,10,0.000000,0.058824,0.058824,0.000000,...,0.058824,0.000000,0.000000,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000


In [ ]:
## cl

In [145]:
latitude = 43.741667
longitude=-79.373333

map_tor_clus=folium.Map(location=[latitude,longitude],zoom_start=11)

x=np.arange(Kclusters)
ys=[i+x+(i*x)**2 for i in range(Kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow =[colors.rgb2hex(i)for i in colors_array]

In [158]:
for lat,lng,poi,cluster in zip(tor_merged['Latitude'],tor_merged['Longitude'],tor_merged['Neighborhood'],tor_merged['Cluster Labels']):
    label=folium.Popup(str(poi)+'Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=15,
        popup=label,
        color=rainbow[-1],
        fill=False,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.10).add_to(map_tor_clus)

map_tor_clus

Lets explore venues in the neighborhood